# Подготовим необходимые библиотеки

In [ ]:
!pip install pytorch_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 868.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-m

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm.auto import tqdm
from pytorch_transformers import AdamW
from IPython.display import clear_output

from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(palette='summer')

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Загрузим файл с данными

In [ ]:
data = pd.read_excel('/content/gdrive/MyDrive/Гилфорд Маг Диплом Розовская/New results/cleaned data Guilford.xlsx')
data

,Unnamed: 0,ID,attempt,n,answer,origNew,prepr_text,simple_category
0,0,PTxGzRMvLY,1,1,сложить из спичек домик,3,сложить из спичек домик,5
1,1,PTxGzRMvLY,1,2,использовать спички как опору чего - нибудь,5,использовать спички как опору чего нибудь,6
2,2,PTxGzRMvLY,1,3,спичками мжно ковыряться в зубах,3,спичками мжно ковыряться в зубах,6
3,3,PTxGzRMvLY,1,4,"если сложить спички в виде колодца, можно испо...",5,если сложить спички в виде колодца можно испол...,6
4,4,PTxGzRMvLY,1,5,"если разукрасить спички, то можно сложить их н...",2,если разукрасить спички то можно сложить их на...,5
...,...,...,...,...,...,...,...,...
7351,9256,2008_2009_320,0,2,для поджога мусора,4,для поджога мусора,1
7352,9257,2008_2009_320,0,3,для играния,3,для играния,7
7353,9258,2008_2009_321,0,1,зубочистка - зубы чистить,3,зубочистка зубы чистить,6
7354,9259,2008_2009_321,0,2,как палочка - для игр,3,как палочка для игр,7


In [ ]:
class TranslationModel:
    def __init__(self, model, tokenizer, from_lang: str, to_lang: str, sent_len: int = 300):
        """
        Initialize the model and tokenizer for translation, specifying source and target languages.
        """
        self.model = model
        self.model.to('cuda')
        self.tokenizer = tokenizer
        # Ensure the tokenizer has a padding token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.from_lang = from_lang
        self.to_lang = to_lang
        self.sent_len = sent_len


    def tokenize(self, sent: str):
        '''Tokenize input sentence'''
        return self.tokenizer(sent, return_tensors='pt', padding=True, truncation=True, max_length=self.sent_len)


    def translate(self, inputs):
        '''
        Generate translation using the model.
        '''
        inputs = inputs.to(self.model.device)
        translated_tokens = self.model.generate(
          **inputs, forced_bos_token_id=self.tokenizer.lang_code_to_id[self.to_lang],
          max_length=self.sent_len)
        return translated_tokens

    def get_decoded(self, toks) -> list:
        '''
        Convert vector tokens into sentences, omitting special tokens.
        '''
        return self.tokenizer.batch_decode(toks, skip_special_tokens=True)

    def flow(self, sentence):
      inputs = self.tokenize(sentence)
      translated_tokens = self.translate(inputs)
      translation = self.get_decoded(translated_tokens)[0]
      return translation

# Используется модель с huggingface для перевода

In [ ]:
model_name = 'facebook/nllb-200-distilled-600M'
from_lang = "rus_Cyrl"
to_lang = "eng_Latn"
rus_tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang=from_lang)
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
translator = TranslationModel(nllb_model, rus_tokenizer, from_lang="ru", to_lang="eng_Latn")

In [ ]:
translations = []

for index, row in tqdm(data.iterrows()):
  t = translator.flow(row['prepr_text'])
  translations.append(t)

#print(translations[1100:1110])

0it [00:00, ?it/s]

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


['brush to clean small holes', 'lock to close access to the lock well', 'Wrapping the cotton to clean the ears and nose', 'Playing puzzles with a friend', 'Write something soaking a tip in water.', 'Use as a toothbrush by sharpening the end with a knife', 'I use the stomach lumps to make the work.', 'I can get on the couch and eat with them.', 'folding the house with glue and matches', 'pencil to write on paper with a wet match']


In [ ]:
translations = [t.strip('.') for t in translations]
print(translations[1100:1110])

['brush to clean small holes', 'lock to close access to the lock well', 'Wrapping the cotton to clean the ears and nose', 'Playing puzzles with a friend', 'Write something soaking a tip in water', 'Use as a toothbrush by sharpening the end with a knife', 'I use the stomach lumps to make the work', 'I can get on the couch and eat with them', 'folding the house with glue and matches', 'pencil to write on paper with a wet match']


In [ ]:
data['eng_text'] = translations
data.to_excel('/content/gdrive/MyDrive/Гилфорд Маг Диплом Розовская/New results/translated data Guilford.xlsx')